# **Libraries We'll Be Using**

In [1]:
import numpy as np
import h5py

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import accuracy_score, f1_score

from keras import regularizers
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, BatchNormalization
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.core import Activation

from keras.optimizers import RMSprop

Using TensorFlow backend.


# **Loading Preprocessed Dataset**

In [3]:
# open our uploaded file
svhn_data = h5py.File('SVHN_Preprocessed_Single.h5', 'r')

# load the training, testing and validation set
X_train = svhn_data['X_train'][:]
y_train = svhn_data['y_train'][:]
X_test = svhn_data['X_test'][:]
y_test = svhn_data['y_test'][:]
X_val = svhn_data['X_val'][:]
y_val = svhn_data['y_val'][:]

# close the file
svhn_data.close()


# check that our datasets are correct
print('Training X Shape: ', X_train.shape)
print('Training Y Shape: ', y_train.shape)
print('Testing X Shape: ', X_test.shape)
print('Testing Y Shape: ', y_test.shape)
print('Validation X Shape: ', X_val.shape)
print('Validation Y Shape: ', y_val.shape)

Training X Shape:  (58605, 32, 32, 1)
Training Y Shape:  (58605, 10)
Testing X Shape:  (26032, 32, 32, 1)
Testing Y Shape:  (26032, 10)
Validation X Shape:  (14652, 32, 32, 1)
Validation Y Shape:  (14652, 10)


# **Create a CNN Model**


**Single-Output Model**

In [0]:
def cnn_model_single():

  weight_decay = 1e-4

  model = Sequential()

  # LAYER 1
  model.add(Conv2D(32, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay), 
                   input_shape=X_train.shape[1:]))
  model.add(Activation('elu'))
  model.add(BatchNormalization())
  model.add(Conv2D(32, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
  model.add(Activation('elu'))
  model.add(BatchNormalization())
  model.add(MaxPooling2D(pool_size=(2,2)))
  model.add(Dropout(0.2))


  # LAYER 2
  model.add(Conv2D(64, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
  model.add(Activation('elu'))
  model.add(BatchNormalization())
  model.add(Conv2D(64, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
  model.add(Activation('elu'))
  model.add(BatchNormalization())
  model.add(MaxPooling2D(pool_size=(2,2)))
  model.add(Dropout(0.3))


  # LAYER 3
  model.add(Conv2D(128, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
  model.add(Activation('elu'))
  model.add(BatchNormalization())
  model.add(Conv2D(128, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
  model.add(Activation('elu'))
  model.add(BatchNormalization())
  model.add(MaxPooling2D(pool_size=(2,2)))
  model.add(Dropout(0.4))


  # final layer has 11 activation layers
  # classes for 0-9 and a class for no digits
  model.add(Flatten())
  model.add(Dense(10, activation='softmax'))

  return model

**Compile, train, and save our CNN model**

In [0]:
# create and compile our model
model = cnn_model_single()
model.compile(loss='categorical_crossentropy', 
              optimizer = RMSprop(lr=0.001, decay=1e-6), 
              metrics=["accuracy"])


# parameters for model fitting
batch_size = 128
epochs = 5

# train the model
model.fit(x=X_train, y=y_train,
          validation_data=(X_val, y_val),
          batch_size=batch_size,
          epochs=epochs,
          verbose=1)


# save our model
model.save("SVHN_model_single.h5")

**Check Model Accuracy**

In [4]:
from keras.models import load_model
model = load_model("SVHN_model_single.h5")

# display final accuracy on validation set
scores = model.evaluate(X_val, y_val, verbose=0)
print("Validation Accuracy: %.2f%%" % (scores[1]*100))


# use our cnn model to make predictions on the testing set
test_predictions = model.predict(X_test,)

# accuracy score
test_accuracy = accuracy_score(y_test, test_predictions.round())
print("Test Accuracy Score: %.2f%%" % (test_accuracy*100))

# f1 score
test_f1 = f1_score(y_test, test_predictions.round(), average='micro')
print("Test F1 Score: %.2f%%" % (test_f1*100))

Validation Accuracy: 92.62%
Test Accuracy Score: 91.36%
Test F1 Score: 92.87%


**Dsiplay some predictions our model makes**

In [15]:
import random

# randomly select labels from the test dataset and see what the model predicted
for i in random.sample(range(0, len(X_test)), 5):
  print("Actual Label: \t\t", np.argmax(y_test[i]))
  print("Predicted Label: \t", np.argmax(test_predictions[i]))
  print("")

Actual Label: 		 5
Predicted Label: 	 5

Actual Label: 		 0
Predicted Label: 	 0

Actual Label: 		 8
Predicted Label: 	 8

Actual Label: 		 5
Predicted Label: 	 5

Actual Label: 		 6
Predicted Label: 	 6

